In [ ]:
BASE_PATH='../../'

In [17]:
%matplotlib inline
import keras
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import scipy.ndimage
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [18]:
train_labels=pd.read_csv(BASE_PATH+'train.csv',engine='c')

In [19]:
trainGen=ImageDataGenerator(featurewise_center=False, 
                    samplewise_center=False, 
                     featurewise_std_normalization=False, 
                     samplewise_std_normalization=False, 
                     zca_whitening=False, 
                     zca_epsilon=1e-06, 
                     rotation_range=0, 
                     width_shift_range=10, 
                     height_shift_range=10, 
                     brightness_range=None, 
                     shear_range=0.0, 
                     zoom_range=0.0, 
                     channel_shift_range=0.1, 
                     fill_mode='nearest', 
                     cval=0.0, 
                     horizontal_flip=True, 
                     vertical_flip=False, 
                     rescale=None, 
                     preprocessing_function=None, 
                     data_format=None, 
                     validation_split=0.3)

train_generator = trainGen.flow_from_directory(
        BASE_PATH+'train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 15697 images belonging to 5004 classes.


# Build Model

In [22]:
model = Sequential()
# input 224x224x3
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(256,(3,3),activation='relu'))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5004,activation='softmax'))

sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True) #??

model.compile(loss='categorical_crossentropy',optimizer=sgd) #??


In [24]:
train_generator.reset()
model.fit_generator(train_generator,steps_per_epoch=100,epochs=10)

Epoch 1/10
100/100 [==============================] - 136s 1s/step - loss: 8.5671

INFO:plaidml:Analyzing Ops: 191 of 389 operations complete
INFO:plaidml:Analyzing Ops: 205 of 389 operations complete


100/100 [==============================] - 141s 1s/step - loss: 8.4565
Epoch 3/10
100/100 [==============================] - 131s 1s/step - loss: 8.3834

In [25]:
testGen=ImageDataGenerator(featurewise_center=False, 
                    samplewise_center=False, 
                     featurewise_std_normalization=False, 
                     samplewise_std_normalization=False, 
                     zca_whitening=False, 
                     zca_epsilon=1e-06, 
                     rotation_range=0, 
                     width_shift_range=0, 
                     height_shift_range=0, 
                     brightness_range=None, 
                     shear_range=0.0, 
                     zoom_range=0.0, 
                     channel_shift_range=0.0, 
                     fill_mode='nearest', 
                     cval=0.0, 
                     horizontal_flip=False, 
                     vertical_flip=False, 
                     rescale=None, 
                     preprocessing_function=None, 
                     data_format=None, 
                     validation_split=0.0)

test_generator = testGen.flow_from_directory(
        BASE_PATH+'test',
        target_size=(150, 150),
        batch_size=1,
        class_mode=None)

Found 7960 images belonging to 1 classes.


In [26]:
test_generator.reset()
pred=model.predict_generator(test_generator,verbose=1,workers=8)

7960/7960 [==============================] - 122s 15ms/step

In [27]:
test_labels=test_generator.filenames

In [28]:
def getClasses(arr):
    return ' '.join([list(train_generator.class_indices.keys())[x] for x in arr])

In [29]:
submission=pd.DataFrame({'Image':[x.split('/')[1] for x in test_labels],
                        'Id':[getClasses(np.argpartition(x,-5)[-5:]) for x in pred]
                        })

In [30]:
submission.head()

,Id,Image
0,w_9c506f6 w_fd3e556 w_23a388d w_700ebb4 w_778e474,00028a005.jpg
1,w_9c506f6 w_fd3e556 w_23a388d w_700ebb4 w_778e474,000dcf7d8.jpg
2,w_9c506f6 w_fd3e556 w_23a388d w_778e474 w_700ebb4,000e7c7df.jpg
3,w_9c506f6 w_fd3e556 w_700ebb4 w_23a388d w_778e474,0019c34f4.jpg
4,w_9c506f6 w_fd3e556 w_700ebb4 w_778e474 w_23a388d,001a4d292.jpg


In [31]:
len(submission)

7960

In [32]:
submission.to_csv('submission.csv',index=False)

In [ ]:
!kaggle competitions submit humpback-whale-identification -f submission.csv -m "My submission message"